# Simulation of an Electric System based on Thermoelectrics Plants

In [1]:
from src.map import Map2D, GraphMap
from src.utils.gaussianmixture import DailyElectricityConsumptionBimodal
from src.circuits import Circuit, Block
from src.thermoelectrics import Thermoelectric
from src.part import Coils, Boiler, Generator, SteamTurbine
import pandas as pd
import plotly.express as px

from src.simulation_constants import (
    NO_CIRCUITS,
    NO_THERMOELECTRICS,
    MIN_CITIZEN,
    MAX_CITIZEN,
    MAX_DEVIATION_CITIZEN_IN_BLOCK,
    DEMAND_PER_PERSON,
    DEMAND_INDUSTRIALIZATION,
    VARIABILITY_DEMAND_PER_PERSON,
    VARIABILITY_DEMAND_PER_INDUSTRIALIZATION,
    PEAK_CONSUMPTION_MORNING,
    PEAK_CONSUMPTION_EVENING,
    MAX_DEVIATION_MORNING,
    MAX_DEVIATION_EVENING,
    WEIGHT_MORNING,
    WEIGHT_EVENING,
    MIN_BLOCKS_PER_CIRCUIT,
    MAX_BLOCKS_PER_CIRCUIT,
    IMPORTANCE_ALPHA,
    RANDOM,
    DISTANCE_REGULATOR
)


## Initialize bases 

In [2]:
map_2d = Map2D(
    no_circuits=NO_CIRCUITS,
    no_thermoelectrics=NO_THERMOELECTRICS,
)


map_2d.visualize()

In [3]:
graphMap = GraphMap(
    thermoelectric_labels=[f"Th{i}" for i in range(NO_THERMOELECTRICS)],
    circuits_labels=[f"C{i}" for i in range(NO_CIRCUITS)],
    towers_labels=[f"Tw{i}" for i in range(len(map_2d.towers_positions))],
    thermoelectrics_positions=map_2d.thermoelectrics_positions,
    circuits_positions=map_2d.circuits_positions,
    towers_positions=map_2d.towers_positions,
)

graphMap.visualize()

In [4]:
distance_cost_template = graphMap.thermoelectric_generation_cost

# Generate circuits for map

electric_system_circuits_list: list[Circuit] = []
for i in range(NO_CIRCUITS):
    citizen_count = RANDOM.integers(MIN_CITIZEN, MAX_CITIZEN)
    citizen_range = (
        max(citizen_count - MAX_DEVIATION_CITIZEN_IN_BLOCK, 0),
        min(citizen_count + MAX_DEVIATION_CITIZEN_IN_BLOCK, MAX_CITIZEN),
    )

    industrialization = RANDOM.integers(0, DEMAND_INDUSTRIALIZATION) / DEMAND_INDUSTRIALIZATION

    bimodal_consumption = DailyElectricityConsumptionBimodal(
        base_consumption=DEMAND_PER_PERSON * citizen_count
        + DEMAND_INDUSTRIALIZATION * industrialization,
        base_variability=VARIABILITY_DEMAND_PER_PERSON * citizen_count
        + VARIABILITY_DEMAND_PER_INDUSTRIALIZATION * industrialization,
        mean_morning=PEAK_CONSUMPTION_MORNING,
        mean_evening=PEAK_CONSUMPTION_EVENING,
        std_morning=RANDOM.uniform(1.0, MAX_DEVIATION_MORNING),
        std_evening=RANDOM.uniform(1.0, MAX_DEVIATION_EVENING),
        weight_morning=WEIGHT_MORNING,
        weight_evening=WEIGHT_EVENING,
    )

    electric_system_circuits_list.append(
        Circuit(
            graphMap.circuits_nodes[i].id,
            gaussian_mixture=bimodal_consumption,
            blocks_range=(MIN_BLOCKS_PER_CIRCUIT, MAX_BLOCKS_PER_CIRCUIT),
            citizens_range=citizen_range,
            industrialization=industrialization,
        )
    )

# Generate Thermoelectric generation based in the nearest circuits
mapper_circuit_with_thermoelectric = {}

for c in electric_system_circuits_list:

    filtered = [f for f in graphMap.thermoelectric_generation_cost if f[1] == c.id]
    filtered = sorted(filtered, key=lambda x: x[2])
    mapper_circuit_with_thermoelectric[c.id] = filtered[0][0]


import plotly.graph_objects as go

# Create edges for the plotly graph
edges = []
edges_labels = []
for circuit, thermoelectric in mapper_circuit_with_thermoelectric.items():

    circuitPos = [c.position for c in graphMap.circuits_nodes if c.id == circuit][0]
    thermoelectricPos = [
        t.position for t in graphMap.thermoelectrics_nodes if t.id == thermoelectric
    ][0]

    edges.append((circuitPos, thermoelectricPos))
    edges_labels.append(f"{circuit} -> {thermoelectric}")


# Create a plotly graph
fig = go.Figure()

# Add edges to the plotly graph
for i, edge in enumerate(edges):
    circuit, thermoelectric = edge
    fig.add_trace(
        go.Scatter(
            x=[circuit[0], thermoelectric[0]],
            y=[circuit[1], thermoelectric[1]],
            mode="lines",
            line=dict(color="black"),
            name="Wire Connection",
            showlegend=False,
            hoverinfo="text",
            text=edges_labels[i],
            hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell"),
        )
    )

# Add nodes to the plotly graph
all_nodes = graphMap.circuits_nodes + graphMap.thermoelectrics_nodes
for i, node in enumerate(all_nodes):
    fig.add_trace(
        go.Scatter(
            x=[node.position[0]],
            y=[node.position[1]],
            mode="markers+text",
            marker=dict(
                size=10, color="blue" if i >= len(graphMap.circuits_nodes) else "red"
            ),
            text=[node.id],
            textposition="top center",
            hoverinfo="text",
        )
    )

fig.update_layout(
    showlegend=False,
    title="Circuits and Thermoelectrics",
    height=800,
    plot_bgcolor="lightgreen",
)

fig.show()

In [5]:
# Generate the distance matrix


def distance_template_to_distance_matrix(
    template: list[tuple[str, str, float, list[str]]],
    thermoelectrics: list[str],
    circuits: list[str],
):
    matrix = [[-1 for _ in range(len(circuits))] for _ in range(len(thermoelectrics))]

    c_map = {}
    t_map = {}

    for i, t in enumerate(thermoelectrics):
        t_map[t] = i

    for i, c in enumerate(circuits):
        c_map[c] = i

    max_cost = 1
    for t, c, cost, _ in template:
        matrix[t_map[t]][c_map[c]] = cost
        max_cost = max(max_cost, cost)

    for t, c, _, _ in template:
        matrix[t_map[t]][c_map[c]] /= max_cost * DISTANCE_REGULATOR

    return matrix


matrix = distance_template_to_distance_matrix(
    distance_cost_template,
    [t.id for t in graphMap.thermoelectrics_nodes],
    [c.id for c in graphMap.circuits_nodes],
)


distance_matrix_df = pd.DataFrame(
    matrix,
    index=[t.id for t in graphMap.thermoelectrics_nodes],
    columns=[c.id for c in graphMap.circuits_nodes],
)

distance_matrix_df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49
Th0,0.197800,0.264327,0.218818,0.100290,0.163509,0.245941,0.522912,0.591912,0.201555,0.566927,...,0.174180,0.414128,0.102365,0.100316,0.422538,0.670923,0.182230,0.109028,0.658833,0.228573
Th1,0.361334,0.815175,0.769666,0.651137,0.387745,0.796789,0.055039,0.058062,0.424500,0.033077,...,0.725028,0.156006,0.653213,0.458313,0.164870,0.137073,0.373537,0.450719,0.124983,0.779421
Th2,0.475438,0.929279,0.883771,0.765242,0.501849,0.910894,0.169144,0.082302,0.538604,0.098596,...,0.839133,0.270110,0.767317,0.572418,0.278975,0.015043,0.487641,0.564824,0.010166,0.893525
Th3,0.040503,0.434933,0.389425,0.270896,0.007503,0.416548,0.365615,0.434615,0.044258,0.409631,...,0.344787,0.256831,0.272971,0.078072,0.265242,0.513626,0.024934,0.070478,0.501536,0.399179
Th4,0.437347,0.038079,0.029060,0.144981,0.403056,0.019694,0.762459,0.831458,0.441101,0.806474,...,0.067048,0.653674,0.239476,0.339863,0.662085,0.910470,0.421777,0.348575,0.898379,0.012674
Th5,0.364662,0.110890,0.065381,0.072296,0.330372,0.092504,0.689774,0.758774,0.368417,0.733789,...,0.020743,0.580990,0.166791,0.267178,0.589401,0.837785,0.349092,0.275891,0.825695,0.075136
Th6,0.069212,0.523053,0.477544,0.359016,0.095623,0.504667,0.268783,0.337783,0.132378,0.312799,...,0.432906,0.159999,0.361091,0.166191,0.168410,0.416795,0.081415,0.158598,0.404704,0.487299
Th7,0.422202,0.876043,0.830535,0.712006,0.448614,0.857658,0.115908,0.029066,0.485368,0.045360,...,0.785897,0.216874,0.714081,0.519182,0.225739,0.059207,0.434405,0.511588,0.047117,0.840289


In [6]:
# Generate thermoelectrics
electric_system_thermoelectrics_list: list[Thermoelectric] = []

for i, t in enumerate(graphMap.thermoelectrics_nodes):
    generated_thermoelectric_min_cost = 0

    for j, c in enumerate(graphMap.circuits_nodes):
        if mapper_circuit_with_thermoelectric[c.id] == t.id:
            generated_thermoelectric_min_cost += (
                electric_system_circuits_list[j].mock_electric_consume
                + electric_system_circuits_list[j].mock_electric_consume
                * matrix[i][j]
                * 24
            )

    

    electric_system_thermoelectrics_list.append(
        Thermoelectric(
            id=t.id,
            parts=[],
            total_capacity=generated_thermoelectric_min_cost + 200,
        )
    )
    thermoelectric_data = {
        "ID": [t.id for t in electric_system_thermoelectrics_list],
        "Total Capacity": [
            t.total_capacity for t in electric_system_thermoelectrics_list
        ],
    }

    thermoelectric_df = pd.DataFrame(thermoelectric_data)


sorted_electric_system_circuit_list = sorted(
    electric_system_circuits_list,
    key=lambda x: list(
        filter(
            lambda y: mapper_circuit_with_thermoelectric[x.id] == y.id,
            graphMap.thermoelectrics_nodes,
        ))[0].id)

colors = [
    list(
        filter(
            lambda x, y=circuit: mapper_circuit_with_thermoelectric[y.id] == x.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id
    for circuit in sorted_electric_system_circuit_list
]

print(colors)

print([t.id for t in electric_system_thermoelectrics_list])

fig = px.bar(
    thermoelectric_df,
    x="ID",
    y="Total Capacity",
    title="Total Capacity of Thermoelectrics",
    height=800,
    color=[t.id for t in electric_system_thermoelectrics_list],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig.show()

['Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th4', 'Th4', 'Th4', 'Th4', 'Th4', 'Th5', 'Th5', 'Th5', 'Th5', 'Th5', 'Th5', 'Th6', 'Th6', 'Th6', 'Th6', 'Th6', 'Th6', 'Th6', 'Th7', 'Th7', 'Th7']
['Th0', 'Th1', 'Th2', 'Th3', 'Th4', 'Th5', 'Th6', 'Th7']


In [7]:

mock_electric_consume_data = {
    "Circuit ID": [c.id for c in sorted_electric_system_circuit_list],
    "Mock Electric Consume": [c.mock_electric_consume for c in sorted_electric_system_circuit_list],
}

mock_electric_consume_df = pd.DataFrame(mock_electric_consume_data)

fig = px.bar(
    mock_electric_consume_df,
    x="Circuit ID",
    y="Mock Electric Consume",
    title="Electric Consume per Circuit [24 hours]",
    height=800,
    color=colors
)
fig.show()

In [8]:
max_population_of_circuits = -1
max_population_of_block = -1

for circuit in electric_system_circuits_list:
    max_population_of_circuits = max(
        circuit.get_all_block_population(), max_population_of_circuits
    )
    for block in circuit.blocks:
        max_population_of_block = max(block.citizens.amount, max_population_of_block)

auxiliary_data_max_population_of_circuits = max_population_of_circuits
auxiliary_data_max_population_of_block = max_population_of_block


def get_circuit_importance(circuit: Circuit) -> float:
    return (
        circuit.get_all_block_population() / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + circuit.industrialization * (1 - IMPORTANCE_ALPHA)


def get_block_importance(block: Block) -> float:
    return (
        block.citizens.amount / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + block.industrialization * (1 - IMPORTANCE_ALPHA)


def set_importance(ci: list[Circuit]):
    for circuit in ci:
        for block in circuit.blocks:
            block.importance = get_block_importance(block)
        circuit.importance = get_circuit_importance(circuit)

set_importance(electric_system_circuits_list)

max_population_data = {
    "Max Population of Circuits": [auxiliary_data_max_population_of_circuits],
    "Max Population of Blocks": [auxiliary_data_max_population_of_block],
}

max_population_df = pd.DataFrame(max_population_data)

max_population_df

,Max Population of Circuits,Max Population of Blocks
0,19519,4965


In [9]:
# All data of one block of a circuit (importance, citizens, industrialization, demand per hour)

circuit_stats = electric_system_circuits_list[0]

blocks = circuit_stats.blocks

citizens = []
importance = []
demand_per_hour = []

for block in blocks:
    citizens.append(block.citizens.amount)
    importance.append(block.importance)
    demand_per_hour.append(sum(block.predicted_demand_per_hour))

block_stats_df = pd.DataFrame(
    {
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Citizens",
    title=f"Block Citizens in Circuit {circuit_stats.id}",
    height=800,
)

fig_importance = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Importance",
    title=f"Block Importance in Circuit {circuit_stats.id}",
    height=800,
)

fig_demand_per_hour = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Demand per Hour",
    title=f"Block Demand per Hour in Circuit {circuit_stats.id}",
    height=800,
)


fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [10]:
# Compare data from circuits

citizens = []
importance = []
demand_per_hour = []

for circuit in sorted_electric_system_circuit_list:
    
    importance.append(circuit.importance)
    demand_per_hour.append(
        sum([sum(block.predicted_demand_per_hour) for block in circuit.blocks])
    )

# Separate for properties
citizens = [
    sum(block.citizens.amount for block in circuit.blocks)
    for circuit in electric_system_circuits_list
]

circuit_comparison_df = pd.DataFrame(
    {   "Circuit ID": [c.id for c in sorted_electric_system_circuit_list], 
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Citizens",
    title="Total Citizens per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_importance = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Importance",
    title="Importance per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_demand_per_hour = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Demand per Hour",
    title="Total Demand per Circuit in a Day",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [11]:
circuit_id = "C0"
circuit = next(c for c in electric_system_circuits_list if c.id == circuit_id)

block_demand_data = {"Block ID": [], "Hour": [], "Demand": []}

for i, block in enumerate(circuit.blocks):
    for hour, demand in enumerate(block.predicted_demand_per_hour):
        block_demand_data["Block ID"].append(f"Block {i + 1}")
        block_demand_data["Hour"].append(hour)
        block_demand_data["Demand"].append(demand)

block_demand_df = pd.DataFrame(block_demand_data)

fig_block_demand = px.line(
    block_demand_df,
    x="Hour",
    y="Demand",
    color="Block ID",
    title=f"Hourly Demand per Block in Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_block_demand.show()

circuit_demand_data = {
    "Hour": list(range(24)),
    "Demand": [
        sum(block.predicted_demand_per_hour[hour] for block in circuit.blocks)
        for hour in range(24)
    ],
}

circuit_demand_df = pd.DataFrame(circuit_demand_data)

fig_circuit_demand = px.line(
    circuit_demand_df,
    x="Hour",
    y="Demand",
    title=f"Hourly Demand of Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_circuit_demand.show()

# Simulation

In [12]:
from src.people import ThermoelectricAgent, ThermoelectricAgentPerception
from src.worldstate import WorldState


class Simulation:
    def __init__(
        self,
        circuits: list[Circuit],
        thermoelectrics: list[Thermoelectric],
        distance_matrix: list[list[float]],
        get_circuit_importance: callable,
        get_block_importance: callable,
    ):
        self.world_state_manager = WorldState(
            circuits=circuits,
            thermoelectrics=thermoelectrics,
            distance_matrix=distance_matrix,
            get_block_importance=get_block_importance,
            get_circuit_importance=get_circuit_importance,
        )

        self.thermoelectric_agents: list[ThermoelectricAgent] = []

    def create_thermoelectric_agent_perception(
        self, thermoelectric
    ) -> ThermoelectricAgentPerception:

        return ThermoelectricAgentPerception(
            thermoelectric=thermoelectric,
            general_deficit=self.world_state_manager.general_deficit,
            general_demand=self.world_state_manager.general_demand,
            general_offer=self.world_state_manager.general_offer,
        )

    def simulate(self, simulation_days=50):

        for thermoelectric in self.world_state_manager.thermoelectrics:
            agent_initial_perception = self.create_thermoelectric_agent_perception(thermoelectric=thermoelectric)
            print("agent perception")
            print(agent_initial_perception)
            thermoelectric_agent = ThermoelectricAgent(
                name=f"Agent of {thermoelectric.id}",
                thermoelectric=thermoelectric,
                perception=agent_initial_perception,
            )

            self.thermoelectric_agents.append(thermoelectric_agent)

    # check progress

    # for agent in self.thermoelectric_agents:
    #     print(agent)


simulation = Simulation(
    circuits=electric_system_circuits_list,
    thermoelectrics=electric_system_thermoelectrics_list,
    distance_matrix=matrix,
    get_circuit_importance=get_circuit_importance,
    get_block_importance=get_block_importance,
)

simulation.simulate()

agent perception
[]
[]
{
            "thermoelectric: {'id': 'Th0', 'total_capacity': 25131.181872518096, 'current_capacity': 0, 'stored_energy': 0, 'parts': []},
            "plant_status": False,
            "parts_status": [],
            "broken_parts": [],
            "max_capacity": 25131.181872518096,
            "current_capacity": 0,
            "power_output_reduction_on_part_failure": [],
            "general_deficit": 0,
            "general_demand": 0,
            "general_offer": 0,
        }
agent perception
[]
[]
{
            "thermoelectric: {'id': 'Th1', 'total_capacity': 48219.88507110203, 'current_capacity': 0, 'stored_energy': 0, 'parts': []},
            "plant_status": False,
            "parts_status": [],
            "broken_parts": [],
            "max_capacity": 48219.88507110203,
            "current_capacity": 0,
            "power_output_reduction_on_part_failure": [],
            "general_deficit": 0,
            "general_demand": 0,
            "general